In [1]:
import dask.dataframe as dd
import datetime as dt
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from pathlib import Path
import gensim

/home/chloezh/miniconda3/envs/pytorch_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
tbl = pd.read_csv('mimic_cxr_note_chexpert.csv')

In [41]:
image_index = list(tbl.index)

In [4]:
label_columns = ['Atelectasis',
                   'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
                   'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding',
                   'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax',
                   'Support Devices']

In [33]:
from sklearn.model_selection import StratifiedShuffleSplit
ss = StratifiedShuffleSplit(n_splits=2, test_size=0.20, random_state=42)

In [35]:
labels = tbl[label_columns]
labels = labels.fillna(0)
labels = labels.replace(-1, 0)
tbl['level'] = np.argmax(labels.to_numpy(), axis=1)

In [37]:
index = list(ss.split(tbl.iloc[:,:-1], tbl.iloc[:,-1]))[0]

In [53]:
index[0]

array([191349, 170553,  40403, ...,  75439,  74488, 110503])

In [45]:
train_tbl = tbl.iloc[index[0],:]
test_tbl = tbl.iloc[index[1],:]

In [54]:
train_tbl.shape[0] + test_tbl.shape[0]  == tbl.shape[0]

True

In [47]:
train_tbl['level'].value_counts()

8     64894
0     48074
1     26070
7     17682
3      9790
9      6754
2      5448
6      3043
12     2878
11     2686
4      2448
13     2418
5      2134
10      340
Name: level, dtype: int64

In [48]:
test_tbl['level'].value_counts()

8     16223
0     12019
1      6517
7      4421
3      2448
9      1689
2      1362
6       761
12      720
11      671
4       612
13      604
5       533
10       85
Name: level, dtype: int64

In [50]:
train_cxr_note = pd.read_csv('train_cxr_note.csv')

In [51]:
train_cxr_note.columns

Index(['dicom_id', 'subject_id', 'study_id',
       'PerformedProcedureStepDescription', 'ViewPosition', 'Rows', 'Columns',
       'StudyDate', 'StudyTime', 'ProcedureCodeSequence_CodeMeaning',
       'ViewCodeSequence_CodeMeaning',
       'PatientOrientationCodeSequence_CodeMeaning', 'note', 'Atelectasis',
       'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
       'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding',
       'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax',
       'Support Devices', 'level'],
      dtype='object')